# Aula 21 - Ensemble & Random Forest


## 1. O que é um ensemble?

![#trabalho em equipe funciona!](https://thumbs.gfycat.com/SlimyTepidAtlanticridleyturtle-size_restricted.gif)

<p style="text-align: center;"> #teamwork </p>



### Vamos ver se isso é verdade mesmo!

In [ ]:
## imports necessarios

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
%matplotlib inline


Criaremos um modelo para predizer pessoas que devem procurar tratamento para saúde mental em empresas de tecnologia.

Vamos ler os dados e analisar o dataset:

In [ ]:
df = pd.read_csv('survey.csv')

> Dataset retirado do [kaggle](https://www.kaggle.com/osmi/mental-health-in-tech-survey/kernels), porém foram filtrados apenas os países Canada, United Kingdom e United States para facilitar a análise

In [ ]:
df.info()

In [ ]:
df.head()

Vemos várias variáveis como objetos e vários *NaN*s, vamos tratá-los.



In [ ]:
# Timestamp:
del df['Timestamp']

### Variáveis com NaN:



In [ ]:
# comments:
df.loc[df['comments'].notnull(), 'comments'].head()

Nao aprendemos NLP... ainda

In [ ]:
del df['comments']

work_interfere:

In [ ]:
df['work_interfere'].value_counts()

In [ ]:
df['work_interfere'].fillna('DontKnow', inplace=True)

self_employed:

In [ ]:
df['self_employed'].value_counts()

In [ ]:
df['self_employed'].fillna('No', inplace=True)

In [ ]:
df['state'].value_counts()

In [ ]:
del df['state']

### Variaveis categoricas

Gender:

In [ ]:
df['Gender'].unique()

Vemos muitas categorias aqui, vamos tratar:

In [ ]:
male_str = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_str = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
female_str = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]

for (row, col) in df.iterrows():

    if str.lower(col.Gender) in male_str:
        df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_str:
        df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in trans_str:
        df['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

In [ ]:
df['Gender'].value_counts()

In [ ]:
df = df[df['Gender']!='p']

In [ ]:
df['Gender'].unique()

Vamos ver quantos países temos:

In [ ]:
df['Country'].value_counts().plot(kind='barh', figsize=(10,7),
                                        color="coral", fontsize=13)

Vamos criar as dummies

In [ ]:
df = pd.get_dummies(df, columns=['Gender', 'Country', 'self_employed', 'family_history', 'treatment', 'work_interfere',
                 'no_employees', 'remote_work', 'tech_company', 'anonymity', 'leave', 'mental_health_consequence',
                 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
                 'mental_vs_physical', 'obs_consequence', 'benefits', 'care_options', 'wellness_program',
                 'seek_help'])

In [ ]:
df.shape

In [ ]:
df.columns.values

In [ ]:
df.drop(columns=['family_history_No', 'treatment_No', 'remote_work_No', 'tech_company_No'], inplace=True)

Vamos dar uma olhada na distribuićão da variável resposta:

In [ ]:
df['treatment_Yes'].value_counts().plot(kind='bar')

## Finalmente, vamos ao treinamento!

Vamos separar os dados em treino e teste:

In [ ]:
X = df.drop('treatment_Yes', axis=1)
y = df['treatment_Yes']

X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.15, random_state=0)
print(X_train.shape)

 Vamos usar os dois métodos mais dummies que aprendemos até agora para tentar predizer 

## Relembrando Árvores de Decisão:

<img src="https://annalyzin.files.wordpress.com/2016/07/decision-trees-titanic-tutorial.png" alt="drawing" style="width:500px;"/>
<p style="text-align: center;"> Exemplo de árvore de decisão para classificar sobreviventes do Titanic </p>

In [ ]:
# crie o modelo com random state igual a 10
model_tree = DecisionTreeClassifier(random_state=10)
model_tree.fit(X_train, y_train)

In [ ]:
y_pred_tree_train = model_tree.predict_proba(X_train)
print("Acurácia: {:.2f}".format(accuracy_score(y_train, y_pred_tree_train[:,1]>0.5)))
print("Precisão: {:.2f}".format(precision_score(y_train, y_pred_tree_train[:,1]>0.5)))
print("Sensibilidade: {:.2f}".format(recall_score(y_train, y_pred_tree_train[:,1]>0.5)))

In [ ]:
y_pred_tree = model_tree.predict_proba(X_test)
print("Acurácia: {:.2f}".format(accuracy_score(y_test, y_pred_tree[:,1]>0.5)))
print("Precisão: {:.2f}".format(precision_score(y_test, y_pred_tree[:,1]>0.5)))
print("Sensibilidade: {:.2f}".format(recall_score(y_test, y_pred_tree[:,1]>0.5)))


## KNN 

![#knn!](https://importq.files.wordpress.com/2017/11/knn_neigh.gif?w=656)

 - algoritmo de abordagem "preguiçosa"
 - assume que elementos similares estão em proximidade
 - Calcula a distância para os *N* vizinhos mais próximos e determina a classe de acordo com a classe dos vizinhos
 
Mais informações: https://www.analyticsvidhya.com/blog/2018/03/introduction-k-neighbours-algorithm-clustering/

In [ ]:
# crie o modelo com random state igual a 10
model_knn = KNeighborsClassifier()
model_knn.fit(X_train,y_train)

In [ ]:
y_pred_knn_train = model_knn.predict_proba(X_train)
print("Acurácia: {:.2f}".format(accuracy_score(y_train, y_pred_knn_train[:,1]>0.5)))
print("Precisão: {:.2f}".format(precision_score(y_train, y_pred_knn_train[:,1]>0.5)))
print("Sensibilidade: {:.2f}".format(recall_score(y_train, y_pred_knn_train[:,1]>0.5)))

In [ ]:
y_pred_knn = model_knn.predict_proba(X_test)
print("Acurácia: {:.2f}".format(accuracy_score(y_test, y_pred_knn[:,1]>0.5)))
print("Precisão: {:.2f}".format(precision_score(y_test, y_pred_knn[:,1]>0.5)))
print("Sensibilidade: {:.2f}".format(recall_score(y_test, y_pred_knn[:,1]>0.5)))

### Como acham que podemos combinar esses modelos?
- .
- .
- .
- .

In [ ]:
model_vot = VotingClassifier(estimators=[('tree', model_tree), ('knn', model_knn)], voting='soft')
model_vot.fit(X_train, y_train)                             

In [ ]:
y_pred = model_vot.predict(X_test)
print("Acurácia: {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Precisão: {:.2f}".format(precision_score(y_test, y_pred)))
print("Sensibilidade: {:.2f}".format(recall_score(y_test, y_pred)))

In [ ]:
x_novo = np.array([y_pred_tree_train[:,1], y_pred_knn_train[:,1]])
x_novo = x_novo.transpose()

x_novo_test = np.array([y_pred_tree[:,1], y_pred_knn[:,1]])
x_novo_test = x_novo_test.transpose()

In [ ]:
model_lr2 = LogisticRegression()
model_lr2.fit(x_novo, y_train)

In [ ]:
y_pred_lr = model_lr2.predict_proba(x_novo_test)
print("Acurácia: {:.2f}".format(accuracy_score(y_test, y_pred_lr[:,1]>0.5)))
print("Precisão: {:.2f}".format(precision_score(y_test, y_pred_lr[:,1]>0.5)))
print("Sensibilidade: {:.2f}".format(recall_score(y_test, y_pred_lr[:,1]>0.5)))

## 2. Tipos de Ensemble:

Existem vários formas diferentes de combinar os modelos. As principais são:


### **1. Voting Based Classifier:**

**1. a) Majority Vote**

A ideia é fazer uma votação entre as predições dos modelos. A classe que tiver mais votos vence. Também podemos ter uma variação desse algoritmo, o **Weighted Voting Classifier**, em que na votação alguns modelos tem mais peso que outros.

![](voting.png)


**1. b) Average Classifier:**

A ideia é similar ao anterior, porém ao invés de uma votação é calculada a média das predições. Da mesma forma podemos ter alguns modelos com mais peso que outros tendo um **Weighted Average Classifier**
![](average.png)


### 2. Stacking:

Nesse modelo as predições dos modelos anteriores são combinadas por um outro modelo para obter a saída final. Podem ser criadas várias camadas com modelos diferentes.

![](stacking.png)

### 3. Boosting:

Os modelos são treinados com os mesmos datasets, porém os pesos das intâncias são ajustados de acordo com o erro das predições anteriores. Veram mais na próxima aula...

![](boosting.png)

## 4. BAGGING: 
Todos os modelos deste tipo de ensemble são do **mesmo algoritmo**, porém os dados de entrada de cada um são amostras do dado original, com a **mesma quantidade de dados do dataset original**, selecionadas usando o método bootstrap (**aleatória com repetição**). 

**AQUI TEMOS A RANDOM FOREST**

![](bagging.png)

## 4. RANDOM FOREST

###  Como Funciona?

![#OOB!](https://cdn-images-1.medium.com/max/1600/1*yoW30XVqAnKOA-7AArXqNg.gif)

- Algoritmo de bagging que usa árvores de decisão
- Cada árvore terá um conjunto diferente de dados e de features
- Out-of-bag score: os dados que não foram usados naquela árvore são utilizados como teste

In [ ]:
model = RandomForestClassifier(n_estimators=10, random_state=0, oob_score=True)
model.fit(X_train, y_train)

In [ ]:
model.oob_score_

In [ ]:
y_pred = model.predict(X_test)
print("Acurácia: {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Precisão: {:.2f}".format(precision_score(y_test, y_pred)))
print("Sensibilidade: {:.2f}".format(recall_score(y_test, y_pred)))

Vamos testar outras combinações de parâmetros?

`Dica 1: Manter o mesmo random_state para comparação de resultados
 Dica 2: Os parâmetros são similares as árvores de decisão
 Dica 3: Número de estimadores e de features são os mais relevantes ` 

Feature importance:

In [ ]:
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::1]


# Plot the feature importances of the forest
plt.figure(figsize=(15,15))
plt.title("Feature importances")
plt.barh(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.yticks(range(X.shape[1]), X.columns[indices])
plt.ylim([-1, X.shape[1]])
plt.show()

In [ ]:
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(model.estimators_[0], out_file='tree.dot', 
                feature_names = X.columns,
                class_names = ['1', '0'],
                rounded = True, proportion = False, 
                precision = 2, filled = True, max_depth=4)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

### Quais tipos de problemas podemos resolver com random forest?

- .
- .
- .
- .

### Quais vantagens e desvantagens vocês enxergam?

*. Vantagens:
- .
- .
- .
- .

*. Desvantagens:
- .
- .
- .
- .

# 5. LightGBM (Gradient Boosting Machine)


![](https://image.slidesharecdn.com/stratauk17salvarisfierrofastretraining-170523090635/95/speeding-up-machinelearning-applications-with-the-lightgbm-library-6-638.jpg)

In [ ]:
import lightgbm as lgb

In [ ]:
model = lgb.LGBMClassifier(objective = 'binary',
          n_jobs = 3, # Updated from 'nthread'
          silent = True,
          max_depth = 10,
          min_child_samples=5,
          learning_rate=0.1,
          n_estimators=50,
          num_leaves= 8)


model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

print("Acurácia: {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Precisão: {:.2f}".format(precision_score(y_test, y_pred)))
print("Sensibilidade: {:.2f}".format(recall_score(y_test, y_pred)))

### Quais vantagens e desvantagens vocês enxergam?
*. Vantagens:
- .
- .
- .
- .

*. Desvantagens:
- .
- .
- .

## Desafios legais para serem resolvidos com ensemble:
- https://www.kaggle.com/c/avito-demand-prediction/data
- https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting
- https://www.kaggle.com/c/sf-crime
- https://www.kaggle.com/c/house-prices-advanced-regression-techniques